In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [4]:
data = pd.read_csv('/kaggle/input/diamonds/diamonds.csv')
data

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [5]:
data.drop(data.columns[0], axis = 1, inplace = True)

In [6]:
data

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [8]:
y = data['price']
X = data.drop('price', axis = 1)

In [10]:
len(data['cut'].unique())

5

In [15]:
encoder = LabelEncoder()

X['cut'] = encoder.fit_transform(X['cut'])
cut_mappings = {index: label for index, label in enumerate(encoder.classes_)}

X['color'] = encoder.fit_transform(X['color'])
color_mappings = {index: label for index, label in enumerate(encoder.classes_)}

X['clarity'] = encoder.fit_transform(X['clarity'])
clarity_mappings = {index: label for index, label in enumerate(encoder.classes_)}

In [16]:
print(cut_mappings)
print(color_mappings)
print(clarity_mappings)

{0: 'Fair', 1: 'Good', 2: 'Ideal', 3: 'Premium', 4: 'Very Good'}
{0: 'D', 1: 'E', 2: 'F', 3: 'G', 4: 'H', 5: 'I', 6: 'J'}
{0: 'I1', 1: 'IF', 2: 'SI1', 3: 'SI2', 4: 'VS1', 5: 'VS2', 6: 'VVS1', 7: 'VVS2'}


In [20]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8
0,0.006237,0.50,0.166667,0.428571,0.513889,0.230769,0.367784,0.067572,0.076415
1,0.002079,0.75,0.166667,0.285714,0.466667,0.346154,0.362197,0.065195,0.072642
2,0.006237,0.25,0.166667,0.571429,0.386111,0.423077,0.377095,0.069100,0.072642
3,0.018711,0.75,0.833333,0.714286,0.538889,0.288462,0.391061,0.071817,0.082704
4,0.022869,0.25,1.000000,0.428571,0.563889,0.288462,0.404097,0.073854,0.086478
...,...,...,...,...,...,...,...,...,...
53935,0.108108,0.50,0.000000,0.285714,0.494444,0.269231,0.535382,0.097793,0.110063
53936,0.108108,0.25,0.000000,0.285714,0.558333,0.230769,0.529795,0.097623,0.113522
53937,0.103950,1.00,0.000000,0.285714,0.550000,0.326923,0.527002,0.096435,0.111950
53938,0.137214,0.75,0.666667,0.428571,0.500000,0.288462,0.572626,0.103905,0.117610


In [18]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [22]:
std_model = LinearRegression()
l1_model = Lasso(alpha = 1)
l2_model = Ridge(alpha = 1)

std_model.fit(X_train, y_train)
l1_model.fit(X_train, y_train)
l2_model.fit(X_train, y_train)

Ridge(alpha=1)

In [24]:
print(f" Without Regularisation: {std_model.score(X_test, y_test)}")
print(f" Lasso Regularisation: {l1_model.score(X_test, y_test)}")
print(f" Ridge Regularisation: {l2_model.score(X_test, y_test)}")

 Without Regularisation: 0.8815721740243637
 Lasso Regularisation: 0.8817883613073836
 Ridge Regularisation: 0.882052371930459


In [26]:
l2_model = Ridge(alpha = 0.6)
l2_model.fit(X_train, y_train)
print(f" Ridge Regularisation: {l2_model.score(X_test, y_test)}")

 Ridge Regularisation: 0.8819517555764955
